In [1]:
import time

import tenseal as ts
import numpy as np

In [2]:
_MAX_BIT_SIZE_SUM_BY_POLY_MODULUS_DEGREE = {
    1024: 27,
    2048: 54,
    4096: 109,
    8192: 218,
    16384: 438,
    32768: 881,
}

In [3]:
def generate_context(n, bits_scale, int_bits):
    context = ts.context(
        ts.SCHEME_TYPE.CKKS,
        poly_modulus_degree=2**n,
        coeff_mod_bit_sizes=[bits_scale + int_bits, bits_scale, bits_scale + int_bits]
    )

    context.global_scale = pow(2, bits_scale)
    context.generate_galois_keys()

    return context

In [6]:
bits_scale = 29
logN = np.arange(12,14)

d = {}
for n in logN:
    t_list = []
    N = 2**n
    start = time.time()
    context = generate_context(n, bits_scale, 10)
    t_list.append(time.time() - start)

    x = np.random.uniform(low=-1, high=1, size=N//2)
    start = time.time()
    x_enc = ts.ckks_vector(context, x)
    t_list.append(time.time() - start)

    w = np.random.uniform(low=-1, high=1, size=N//2)

    start = time.time()
    y = x_enc.dot(w)
    t_list.append(time.time() - start)

    # --------------

    W = np.random.uniform(low=-1, high=1, size=(N//2, 2**11))

    start = time.time()
    y = x_enc @ W
    t_list.append(time.time() - start)

    d[n] = t_list
    # ---------------

print(f"{'logN':>6s} {'context':>11s} {'encrpyt':>11s} {'dot':>11s} {'mvp':>11s}")
for k, v in d.items():
    context, encrypt, dot, mvp = v
    print(f"{k:>6d} {context:>11.6f} {encrypt:>11.6f} {dot:>11.6f} {mvp:>11.6f}")

  logN     context     encrpyt         dot         mvp
    12    0.140169    0.002988    0.008001    4.495931
    13    0.294620    0.006030    0.019969   30.055524


In [7]:
%timeit generate_context(14, 20, 10)

474 ms ± 5.41 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [ ]:
"""
  logN     context     encrpyt         dot         mvp
    12    0.140662    0.000000    0.015586    3.480678
    13    0.279365    0.015622    0.015652   17.753906
    14    0.562500    0.015626    0.046874   92.052080
    15    1.064275    0.031298    0.078134  378.988804

# dot product (1000 iterations)
logN =  12:    0.005821
logN =  13:    0.013553
logN =  14:    0.031969
logN =  15:    0.074184
"""